<a href="https://colab.research.google.com/github/lucasvazelle/nexialog_challlenge/blob/main/analyse_exploratoire_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pyarrow
from pyspark.sql import SparkSession
from google.colab import drive
import numpy as np


pd.set_option('display.max_columns', None)

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


Si vous voulez utiliser les même chemin que moi, créez un raccourcit dans votre "mon drive" de mon dossier partagé "nexialog"
https://drive.google.com/drive/folders/1d-uDDNs_H3Y977kY4F9A0elEoUKUxyz6?usp=drive_link

In [3]:
chemin_donnees_brutes = "/content/drive/MyDrive/nexialog/250203_tests_fixe_dns_sah_202412_202501.parquet"

In [4]:
chemin_donnees_preprocessees = "/content/drive/MyDrive/nexialog/250203_tests_fixe_dns_sah_202412_202501.parquet"

In [5]:
donnee_brutes = pd.read_parquet(chemin_donnees_brutes, engine="pyarrow").sort_values(by="date_hour")
donnee_brutes.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.462500,0.843750,2.852823,0.550380,86
3812,2024-12-01,64,M22,64_olt_3775,64_peag_1886,BU134,dsp_25,None,33_bdx,5,5.758650,0.338825,3,10.687500,0.000000,3.450000,0.000000,8
3811,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,9.140625,0.281250,3.467647,0.000000,21
3810,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20
3809,2024-12-01,64,M22,64_olt_3772,64_peag_1883,BU1160,dsp_1,33_pebib_32,33_bdx,10,19.316250,26.348017,2,15.187500,0.000000,2.550000,0.000000,12


# rapport des données

In [6]:
! pip install ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.5/393.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=b7eeb959e7c5d35e1795141090c666f5d047114c374173e785ea101e3fbadb5a
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


In [7]:
from ydata_profiling import ProfileReport
profil = ProfileReport(donnee_brutes)  # explorative=True active toutes les analyses


In [8]:
# profil.to_file("rapport_données_brutes.html")

# EDA

In [9]:
donnee_brutes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9964847 entries, 0 to 9964846
Data columns (total 18 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date_hour            datetime64[ns]
 1   code_departement     object        
 2   olt_model            object        
 3   olt_name             object        
 4   peag_nro             object        
 5   boucle               object        
 6   dsp                  object        
 7   pebib                object        
 8   pop_dns              object        
 9   nb_test_dns          int64         
 10  avg_dns_time         float64       
 11  std_dns_time         float64       
 12  nb_test_scoring      int64         
 13  avg_latence_scoring  float64       
 14  std_latence_scoring  float64       
 15  avg_score_scoring    float64       
 16  std_score_scoring    float64       
 17  nb_client_total      int64         
dtypes: datetime64[ns](1), float64(6), int64(3), object(8)
memory usage:

In [10]:
donnee_brutes.describe()

,date_hour,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total
count,9964847,9.964847e+06,9.182437e+06,9.029369e+06,9.964847e+06,9.110649e+06,7.620775e+06,9.110628e+06,7.620755e+06,9.964847e+06
mean,2024-12-31 17:46:54.609195520,1.652946e+01,5.401405e+00,6.829364e-01,8.803355e+00,9.436212e+00,5.829885e-01,3.245861e+00,2.736377e-01,2.533281e+01
min,2024-12-01 00:00:00,0.000000e+00,6.637500e-01,0.000000e+00,0.000000e+00,3.375000e+00,0.000000e+00,7.500000e-01,0.000000e+00,1.000000e+00
25%,2024-12-16 05:00:00,3.000000e+00,3.286479e+00,3.099557e-01,2.000000e+00,7.640625e+00,2.812500e-01,3.112185e+00,4.556381e-03,5.000000e+00
50%,2024-12-31 06:00:00,8.000000e+00,4.658625e+00,4.405936e-01,4.000000e+00,9.337500e+00,3.557562e-01,3.432353e+00,5.068742e-02,1.200000e+01
75%,2025-01-16 08:00:00,1.800000e+01,6.580958e+00,6.926338e-01,1.000000e+01,1.105312e+01,5.031153e-01,3.467647e+00,4.738490e-01,2.800000e+01
max,2025-01-31 23:00:00,1.275000e+03,3.239557e+02,3.675479e+02,2.050000e+02,2.129625e+03,1.507031e+03,3.564706e+00,1.984058e+00,1.330000e+03
std,NaN,2.604615e+01,3.428913e+00,2.228742e+00,1.323398e+01,3.520609e+00,2.698877e+00,3.606079e-01,3.294620e-01,3.861231e+01


In [11]:
len(donnee_brutes)

9964847

In [12]:






len(donnee_brutes.drop_duplicates(["date_hour","olt_name"]))

8492270

In [13]:
len(donnee_brutes.drop_duplicates(["date_hour","peag_nro"]))

4447318

In [14]:
#
len(donnee_brutes[donnee_brutes['nb_client_total'] < 10])


4167808

# Premiers traitements


### BOX → PM → BOUCLE → PAEG → OLT → PEU → PEBIB → SERVEUR

In [15]:
donnee_brutes.columns

Index(['date_hour', 'code_departement', 'olt_model', 'olt_name', 'peag_nro',
       'boucle', 'dsp', 'pebib', 'pop_dns', 'nb_test_dns', 'avg_dns_time',
       'std_dns_time', 'nb_test_scoring', 'avg_latence_scoring',
       'std_latence_scoring', 'avg_score_scoring', 'std_score_scoring',
       'nb_client_total'],
      dtype='object')

In [16]:
# ____________________________________________________________calcul feature pour avoir le moment de la journée
def definir_moment_journee(heure):
    if 0 <= heure < 6:
        return "nuit"
    elif 6 <= heure < 12:
        return "matin"
    elif 12 <= heure < 18:
        return "après-midi"
    else:
        return "soir"

donnee_brutes["moment_journee"] = donnee_brutes["date_hour"].dt.hour.apply(definir_moment_journee)

print(donnee_brutes[["date_hour", "moment_journee"]].head())

      date_hour moment_journee
0    2024-12-01           nuit
3812 2024-12-01           nuit
3811 2024-12-01           nuit
3810 2024-12-01           nuit
3809 2024-12-01           nuit


In [17]:
donnee_brutes.head(2)

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.4625,0.84375,2.852823,0.55038,86,nuit
3812,2024-12-01,64,M22,64_olt_3775,64_peag_1886,BU134,dsp_25,None,33_bdx,5,5.758650,0.338825,3,10.6875,0.00000,3.450000,0.00000,8,nuit


In [18]:
#_____________________________________________ création features si le test dns est au même département que le PM_______________________________

departement_region = {
    # Auvergne-Rhône-Alpes
    "01": "Auvergne-Rhône-Alpes", "03": "Auvergne-Rhône-Alpes", "07": "Auvergne-Rhône-Alpes",
    "15": "Auvergne-Rhône-Alpes", "26": "Auvergne-Rhône-Alpes", "38": "Auvergne-Rhône-Alpes",
    "42": "Auvergne-Rhône-Alpes", "43": "Auvergne-Rhône-Alpes", "63": "Auvergne-Rhône-Alpes",
    "69": "Auvergne-Rhône-Alpes", "73": "Auvergne-Rhône-Alpes", "74": "Auvergne-Rhône-Alpes",
    # Bourgogne-Franche-Comté
    "21": "Bourgogne-Franche-Comté", "25": "Bourgogne-Franche-Comté", "39": "Bourgogne-Franche-Comté",
    "58": "Bourgogne-Franche-Comté", "70": "Bourgogne-Franche-Comté", "71": "Bourgogne-Franche-Comté",
    "89": "Bourgogne-Franche-Comté", "90": "Bourgogne-Franche-Comté",
    # Bretagne
    "22": "Bretagne", "29": "Bretagne", "35": "Bretagne", "56": "Bretagne",
    # Centre-Val de Loire
    "18": "Centre-Val de Loire", "28": "Centre-Val de Loire", "36": "Centre-Val de Loire",
    "37": "Centre-Val de Loire", "41": "Centre-Val de Loire", "45": "Centre-Val de Loire",
    # Corse
    "2A": "Corse", "2B": "Corse",
    # Grand Est
    "08": "Grand Est", "10": "Grand Est", "51": "Grand Est", "52": "Grand Est",
    "54": "Grand Est", "55": "Grand Est", "57": "Grand Est", "67": "Grand Est",
    "68": "Grand Est", "88": "Grand Est",
    # Hauts-de-France
    "02": "Hauts-de-France", "59": "Hauts-de-France", "60": "Hauts-de-France",
    "62": "Hauts-de-France", "80": "Hauts-de-France",
    # Île-de-France
    "75": "Île-de-France", "77": "Île-de-France", "78": "Île-de-France",
    "91": "Île-de-France", "92": "Île-de-France", "93": "Île-de-France",
    "94": "Île-de-France", "95": "Île-de-France",
    # Normandie
    "14": "Normandie", "27": "Normandie", "50": "Normandie",
    "61": "Normandie", "76": "Normandie",
    # Nouvelle-Aquitaine
    "16": "Nouvelle-Aquitaine", "17": "Nouvelle-Aquitaine", "19": "Nouvelle-Aquitaine",
    "23": "Nouvelle-Aquitaine", "24": "Nouvelle-Aquitaine", "33": "Nouvelle-Aquitaine",
    "40": "Nouvelle-Aquitaine", "47": "Nouvelle-Aquitaine", "64": "Nouvelle-Aquitaine",
    "79": "Nouvelle-Aquitaine", "86": "Nouvelle-Aquitaine", "87": "Nouvelle-Aquitaine",
    # Occitanie
    "09": "Occitanie", "11": "Occitanie", "12": "Occitanie", "30": "Occitanie",
    "31": "Occitanie", "32": "Occitanie", "34": "Occitanie", "46": "Occitanie",
    "48": "Occitanie", "65": "Occitanie", "66": "Occitanie", "81": "Occitanie",
    "82": "Occitanie",
    # Pays de la Loire
    "44": "Pays de la Loire", "49": "Pays de la Loire", "53": "Pays de la Loire",
    "72": "Pays de la Loire", "85": "Pays de la Loire",
    # Provence-Alpes-Côte d'Azur
    "04": "Provence-Alpes-Côte d'Azur", "05": "Provence-Alpes-Côte d'Azur",
    "06": "Provence-Alpes-Côte d'Azur", "13": "Provence-Alpes-Côte d'Azur",
    "83": "Provence-Alpes-Côte d'Azur", "84": "Provence-Alpes-Côte d'Azur",
    # Départements d'outre-mer
    "971": "Guadeloupe", "972": "Martinique", "973": "Guyane", "974": "La Réunion", "976": "Mayotte"
}

donnee_brutes["dep_pop_dns"] = donnee_brutes["pop_dns"].str[:2]

# Vérifier si les départements sont égaux
donnee_brutes["depts_egaux"] = donnee_brutes["dep_pop_dns"] == donnee_brutes["code_departement"]

# Vérifier si les départements appartiennent à la même région
donnee_brutes["region_pop_dns"] = donnee_brutes["dep_pop_dns"].map(departement_region)
donnee_brutes["region_pm"] = donnee_brutes["code_departement"].map(departement_region)
donnee_brutes["meme_region"] = donnee_brutes["region_pop_dns"] == donnee_brutes["region_pm"]

# Afficher un aperçu
print(donnee_brutes[["code_departement", "pop_dns", "dep_pop_dns", "depts_egaux", "meme_region"]].head())

     code_departement  pop_dns dep_pop_dns  depts_egaux  meme_region
0                  01  69_lyon          69        False         True
3812               64   33_bdx          33        False         True
3811               64   33_bdx          33        False         True
3810               64   33_bdx          33        False         True
3809               64   33_bdx          33        False         True


In [19]:
donnee_brutes.head(2)

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,dep_pop_dns,depts_egaux,region_pop_dns,region_pm,meme_region
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.4625,0.84375,2.852823,0.55038,86,nuit,69,False,Auvergne-Rhône-Alpes,Auvergne-Rhône-Alpes,True
3812,2024-12-01,64,M22,64_olt_3775,64_peag_1886,BU134,dsp_25,None,33_bdx,5,5.758650,0.338825,3,10.6875,0.00000,3.450000,0.00000,8,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True


In [20]:
#___________________________________ supression ligne si pas beaucoup de client au niveau PM (en moyenne il y a 25 clients par PM)

donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes[donnee_brutes['nb_client_total'] > 15]
assert len(donnee_brutes_sans_pm_avec_peu_de_clients.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))

### on supprime les PM qui ont 15 clients ou moins

In [21]:
# _________________________________________supression ligne si pas beaucoup de test au niveau des PM

seuil_nombre_de_test_minimum = 5

# Suppression des lignes où les deux colonnes sont en dessous du seuil
donnee_brutes_sans_pm_avec_peu_de_clients = donnee_brutes_sans_pm_avec_peu_de_clients[
    ~((donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum) &
      (donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum))
]
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_dns"] < seuil_nombre_de_test_minimum,
    ["avg_dns_time", "std_dns_time"]
] = np.nan

# Remplacer par NaN si nb_test_scoring est sous le seuil
donnee_brutes_sans_pm_avec_peu_de_clients.loc[
    donnee_brutes_sans_pm_avec_peu_de_clients["nb_test_scoring"] < seuil_nombre_de_test_minimum,
    [ "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
] = np.nan

donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests = donnee_brutes_sans_pm_avec_peu_de_clients

# Afficher un aperçu
print(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[
    ["nb_test_scoring", "nb_test_dns", "avg_dns_time", "std_dns_time",
     "avg_latence_scoring", "std_latence_scoring", "avg_score_scoring", "std_score_scoring"]
].head())

assert len(donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.drop_duplicates(["date_hour"])) ==  len(donnee_brutes.drop_duplicates(["date_hour"]))

      nb_test_scoring  nb_test_dns  avg_dns_time  std_dns_time  \
0                  25           61      4.888439      0.834768   
3811                4           17      3.583456      0.252478   
3810               10           10      3.638700      0.616376   
3802               12           20      4.808550      0.810622   
3800               13           27      4.637250      0.434874   

      avg_latence_scoring  std_latence_scoring  avg_score_scoring  \
0               10.462500             0.843750           2.852823   
3811                  NaN                  NaN                NaN   
3810             9.112500             0.237171           3.465882   
3802             9.796875             0.289647           3.404412   
3800             9.951923             0.270216           3.461539   

      std_score_scoring  
0              0.550380  
3811                NaN  
3810           0.005581  
3802           0.207982  
3800           0.004239  


In [22]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,dep_pop_dns,depts_egaux,region_pop_dns,region_pm,meme_region
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.462500,0.843750,2.852823,0.550380,86,nuit,69,False,Auvergne-Rhône-Alpes,Auvergne-Rhône-Alpes,True
3811,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3810,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3802,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,None,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True
3800,2024-12-01,64,M22,64_olt_3763,64_peag_1874,BU136,dsp_25,None,33_bdx,27,4.637250,0.434874,13,9.951923,0.270216,3.461539,0.004239,40,nuit,33,False,Nouvelle-Aquitaine,Nouvelle-Aquitaine,True


In [24]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.shape

(4271670, 24)

In [25]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[['date_hour', 'code_departement', 'olt_model', 'olt_name', 'peag_nro',
 'boucle', 'dsp', 'pebib', 'pop_dns', 'nb_test_dns', 'avg_dns_time',
'std_dns_time', 'nb_test_scoring', 'avg_latence_scoring',
'std_latence_scoring', 'avg_score_scoring', 'std_score_scoring',
'nb_client_total', 'moment_journee', 'depts_egaux', 'meme_region']]
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.462500,0.843750,2.852823,0.550380,86,nuit,False,True
3811,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,False,True
3810,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,False,True
3802,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,None,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,False,True
3800,2024-12-01,64,M22,64_olt_3763,64_peag_1874,BU136,dsp_25,None,33_bdx,27,4.637250,0.434874,13,9.951923,0.270216,3.461539,0.004239,40,nuit,False,True


### Valeurs Manquantes

In [26]:
missing_values = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.isna().sum()
missing_percentage = (donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.isna().mean() * 100).round(2)

missing_df = pd.DataFrame({'Valeurs Manquantes': missing_values, 'Pourcentage (%)': missing_percentage})
missing_df

,Valeurs Manquantes,Pourcentage (%)
date_hour,0,0.00
code_departement,0,0.00
olt_model,3334,0.08
olt_name,0,0.00
peag_nro,0,0.00
boucle,0,0.00
dsp,0,0.00
pebib,1953537,45.73
pop_dns,2,0.00
nb_test_dns,0,0.00


#### **olt_model** (0.08% manquantes)




**Pour traiter les valeurs manquantes de la variable olt_model j'ai fait une imputation basée sur la géographie du réseau. Après avoir identifié que les 3334 valeurs manquantes sont concentrées sur seulement 5 OLT spécifiques et qu'aucun de ces équipements n'a de modèle connu dans le dataset, j'ai analysé la distribution des modèles par département -> par exemple le département 35 privilégie le modèle "M11" tandis que les départements 26, 24 et 07 favorisent le modèle "M22" dcp j'ai imputé chaque valeur manquante en utilisant le modèle le plus fréquent dans le département correspondant..**

In [27]:
donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.olt_model.value_counts()

,count
olt_model,
M22,1800088
M13,820211
M11,754543
old0,402576
M24,304857
old1,146790
M20,39271


In [28]:
print(f"Nombre de valeurs manquantes pour olt_model: {donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna().sum()}")

Nombre de valeurs manquantes pour olt_model: 3334


In [29]:
# Filtrer les lignes avec olt_model manquant
missing_olt_model = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna()]
missing_olt_model

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region
1311,2024-12-01 00:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,26_pebib_62,13_abg,78,5.835010,1.271536,50,13.207500,1.219947,3.119470,0.507015,128,nuit,False,False
2067,2024-12-01 00:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,None,35_ren,16,1.583484,0.428641,5,7.762500,0.251558,3.487059,0.003946,21,nuit,True,True
7017,2024-12-01 01:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,26_pebib_62,13_abg,94,5.615585,1.285407,44,13.078125,0.323814,3.059358,0.415081,138,nuit,False,False
7784,2024-12-01 01:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,None,35_ren,13,1.641462,0.337330,10,7.931250,0.319301,3.392647,0.289918,23,nuit,True,True
12769,2024-12-01 02:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,26_pebib_62,13_abg,93,5.638500,1.073475,44,13.103693,0.412809,3.130949,0.513650,137,nuit,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9949473,2025-01-31 21:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,None,35_ren,24,1.657719,0.665938,9,7.312500,0.281250,3.403922,0.270624,33,soir,True,True
9954536,2025-01-31 22:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,26_pebib_62,13_abg,86,5.519642,0.927844,51,13.257353,1.869853,3.243425,0.348685,137,soir,False,False
9955310,2025-01-31 22:00:00,35,None,35_olt_2049,35_peag_1082,BU836,dsp_1,None,35_ren,12,2.341250,1.980337,5,7.875000,0.688919,3.485294,0.010807,17,soir,True,True
9960382,2025-01-31 23:00:00,26,None,26_olt_1301,26_peag_613,BU400,dsp_1,26_pebib_62,13_abg,100,5.894580,1.061992,47,12.865691,0.557797,3.139487,0.389583,147,soir,False,False


In [30]:
# nbre d'olt_name uniques concernés par ces valeurs manquantes
unique_olt_names = missing_olt_model['olt_name'].nunique()
print(f"Nombre d'olt_name uniques avec des modèles manquants: {unique_olt_names}")

Nombre d'olt_name uniques avec des modèles manquants: 5


In [31]:
olt_counts = missing_olt_model['olt_name'].value_counts()
print("\nles olt_name avec le plus de valeurs manquantes:")
print(olt_counts.head(10))


les olt_name avec le plus de valeurs manquantes:
olt_name
26_olt_1301    1632
35_olt_2049    1539
24_olt_1192     157
24_olt_1193       5
07_olt_330        1
Name: count, dtype: int64


In [32]:
# Pour chaque olt_name avec des valeurs manquantes, on vérifie s'il existe des enregistrements avec un modèle connu
olt_with_known_models = 0
olt_without_known_models = 0

for olt_name in missing_olt_model['olt_name'].unique():
    # Cherche des enregistrements pour cet olt_name avec un modèle connu
    known_models = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests[
        (donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_name'] == olt_name) &
        (~donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests['olt_model'].isna())
    ]['olt_model']

    if len(known_models) > 0:
        olt_with_known_models += 1
    else:
        olt_without_known_models += 1

print(f"\nolt_name avec au moins un modèle connu: {olt_with_known_models}")
print(f"olt_name sans aucun modèle connu: {olt_without_known_models}")


olt_name avec au moins un modèle connu: 0
olt_name sans aucun modèle connu: 5


* on ne peut pas utiliser le modèle le plus fréquent pour chaque OLT ici vu que aucun de ces 5 OLT manquants n'a de modèle connu dans le dataset .. on va changer la méthode d'imputation ..

In [33]:
df_imputed = donnee_brutes_sans_pm_avec_peu_de_clients_et_peu_de_tests.copy()

In [34]:
# je vais tester l'imputation basée sur le département ..
df_imputed['dept'] = df_imputed['olt_name'].str.extract(r'^(\d+)_')

print("Premiers départements extraits:")
print(df_imputed[['olt_name', 'dept']].head(10))

Premiers départements extraits:
         olt_name dept
0        01_olt_1   01
3811  64_olt_3774   64
3810  64_olt_3773   64
3802  64_olt_3765   64
3800  64_olt_3763   64
3797  64_olt_3760   64
3796  64_olt_3759   64
3803  64_olt_3766   64
3794  64_olt_3757   64
3816  64_olt_3779   64


In [35]:
# Pour chaque département -> la distribution des modèles
dept_model_distribution = {}
for dept in df_imputed['dept'].unique():
    if pd.isna(dept):
        continue

    # Obtenir la distribution des modèles pour ce département
    model_counts = df_imputed[
        (df_imputed['dept'] == dept) &
        (~df_imputed['olt_model'].isna())
    ]['olt_model'].value_counts()

    # Si le département a des modèles connus
    if not model_counts.empty:
        dept_model_distribution[dept] = model_counts.to_dict()

# Départements qui nous intéressent (ou on a des valeurs manquantes)
for dept in ['26', '35', '24', '07']:
    if dept in dept_model_distribution:
        print(f"Département {dept}: {dept_model_distribution[dept]}")
    else:
        print(f"Département {dept}: Aucun modèle connu")

Département 26: {'M22': 14454, 'M13': 4838, 'M11': 1722, 'old0': 1632, 'M20': 87}
Département 35: {'M11': 24354, 'M22': 21546, 'M24': 7788, 'old0': 6528, 'M13': 6232, 'M20': 174}
Département 24: {'M22': 17367, 'M13': 3264, 'M11': 415, 'M20': 313}
Département 07: {'M22': 7181, 'M13': 1632, 'M11': 114, 'M20': 77}


In [36]:
# Créer un dictionnaire des modèles les plus fréquents par département
dept_model = {}
for dept, models in dept_model_distribution.items():
    # Trouver le modèle avec le plus d'occurrences
    most_common_model = max(models.items(), key=lambda x: x[1])[0]
    dept_model[dept] = most_common_model

In [37]:
# Compter les valeurs manquantes avant imputation
missing_before = df_imputed['olt_model'].isna().sum()
print(f"Valeurs manquantes avant imputation: {missing_before}")

# Le modèle le plus fréquent globalement
default_model = 'M22'

# Imputer les valeurs manquantes
for idx in df_imputed[df_imputed['olt_model'].isna()].index:
    dept = df_imputed.loc[idx, 'dept']
    if dept in dept_model:
        # le modèle le plus fréquent du département
        df_imputed.loc[idx, 'olt_model'] = dept_model[dept]
    else:
        # le modèle par défaut
        df_imputed.loc[idx, 'olt_model'] = default_model

print(f"Valeurs manquantes finales dans olt_model: {df_imputed['olt_model'].isna().sum()}")

Valeurs manquantes avant imputation: 3334
Valeurs manquantes finales dans olt_model: 0


#### pop_dns (2 valeurs manquantes)

**pour l'imputation des valeurs manquantes de pop_dns j'ai cherché d'abord la valeur la plus fréquente pour la combinaison exacte d'OLT et PEAG, car les équipements partageant le même chemin réseau utilisent généralement le même serveur DNS.. Si cette information n'est pas disponible, on impute par la valeur au niveau de l'OLT seul. En dernier recours, nous utilisons la valeur la plus fréquente dans l'ensemble du réseau..**

In [38]:
df_imputed.pop_dns.value_counts()

,count
pop_dns,
94_cbv,796160
77_mit,764640
33_bdx,639589
69_lyon,576913
13_abg,550793
35_ren,503133
67_sbg,274478
59_lil,165962


In [39]:
# lignes avec pop_dns manquant
missing_pop_dns = df_imputed[df_imputed['pop_dns'].isna()]
missing_pop_dns

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region,dept
5632414,2025-01-04 04:00:00,60,M13,60_olt_3511,60_peag_1702,BU334,dsp_1,None,None,0,NaN,NaN,17,5.525735,0.409279,3.252249,0.705607,17,nuit,False,False,60
7100651,2025-01-13 16:00:00,75,M13,75_olt_4505,75_peag_2301,BU270,dsp_1,78_pebib_31,None,0,NaN,NaN,20,4.865625,0.555799,3.000882,1.111764,20,après-midi,False,False,75


In [40]:
# Créer dictionnaires de valeurs les plus fréquentes par niveau de granularité
pop_by_olt_peag = df_imputed.dropna(subset=['pop_dns']).groupby(['olt_name', 'peag_nro'])['pop_dns'].apply(
    lambda x: x.value_counts().index[0] if len(x) > 0 else None
).to_dict()

pop_by_olt = df_imputed.dropna(subset=['pop_dns']).groupby('olt_name')['pop_dns'].apply(
    lambda x: x.value_counts().index[0] if len(x) > 0 else None
).to_dict()

default_pop = df_imputed['pop_dns'].mode()[0]

# Imputer les valeurs manquantes
for idx in missing_pop_dns.index:
    olt = df_imputed.loc[idx, 'olt_name']
    peag = df_imputed.loc[idx, 'peag_nro']

    # Chercher la valeur à imputer par ordre de spécificité
    if (olt, peag) in pop_by_olt_peag:
        df_imputed.loc[idx, 'pop_dns'] = pop_by_olt_peag[(olt, peag)]
    elif olt in pop_by_olt:
        df_imputed.loc[idx, 'pop_dns'] = pop_by_olt[olt]
    else:
        df_imputed.loc[idx, 'pop_dns'] = default_pop

In [41]:
# les valeurs manquantes de avg_dns_time correspondent à nb_test_dns = 0 ???
missing_avg_dns = df_imputed['avg_dns_time'].isna()
zero_dns_tests = df_imputed['nb_test_dns'] == 0
total_missing_avg_dns = missing_avg_dns.sum()
total_zero_dns_tests = zero_dns_tests.sum()

print(f"Nombre total de valeurs manquantes pour avg_dns_time: {total_missing_avg_dns}")
print(f"Nombre total de lignes avec nb_test_dns = 0: {total_zero_dns_tests}")

Nombre total de valeurs manquantes pour avg_dns_time: 2959
Nombre total de lignes avec nb_test_dns = 0: 2


* on a opté à garder les nans ici et creer une variable indicatrice pour les moyennes et variance de chaque tests pour indiquer si y a des valeurs manquantes ou pas (ça va nous aider au moment de la construction des modeles pour les OLT)

In [43]:
# Création de variables indicatrices pour les tests DNS
df_imputed['missing_avg_dns_time'] = df_imputed['avg_dns_time'].isna().astype(int)
df_imputed['missing_std_dns_time'] = df_imputed['std_dns_time'].isna().astype(int)

# Création de variables indicatrices pour les tests de scoring
df_imputed['missing_avg_latence_scoring'] = df_imputed['avg_latence_scoring'].isna().astype(int)
df_imputed['missing_std_latence_scoring'] = df_imputed['std_latence_scoring'].isna().astype(int)
df_imputed['missing_avg_score_scoring'] = df_imputed['avg_score_scoring'].isna().astype(int)
df_imputed['missing_std_score_scoring'] = df_imputed['std_score_scoring'].isna().astype(int)

In [44]:
df_imputed.head()

,date_hour,code_departement,olt_model,olt_name,peag_nro,boucle,dsp,pebib,pop_dns,nb_test_dns,avg_dns_time,std_dns_time,nb_test_scoring,avg_latence_scoring,std_latence_scoring,avg_score_scoring,std_score_scoring,nb_client_total,moment_journee,depts_egaux,meme_region,dept,missing_avg_dns_time,missing_std_dns_time,missing_avg_latence_scoring,missing_std_latence_scoring,missing_avg_score_scoring,missing_std_score_scoring
0,2024-12-01,01,old0,01_olt_1,01_peag_1,BU966,dsp_1,69_pebib_1,69_lyon,61,4.888439,0.834768,25,10.462500,0.843750,2.852823,0.550380,86,nuit,False,True,01,0,0,0,0,0,0
3811,2024-12-01,64,M22,64_olt_3774,64_peag_1885,BU112,dsp_25,None,33_bdx,17,3.583456,0.252478,4,NaN,NaN,NaN,NaN,21,nuit,False,True,64,0,0,1,1,1,1
3810,2024-12-01,64,M22,64_olt_3773,64_peag_1884,BU124,dsp_25,None,33_bdx,10,3.638700,0.616376,10,9.112500,0.237171,3.465882,0.005581,20,nuit,False,True,64,0,0,0,0,0,0
3802,2024-12-01,64,M22,64_olt_3765,64_peag_1876,BU136,dsp_25,None,33_bdx,20,4.808550,0.810622,12,9.796875,0.289647,3.404412,0.207982,32,nuit,False,True,64,0,0,0,0,0,0
3800,2024-12-01,64,M22,64_olt_3763,64_peag_1874,BU136,dsp_25,None,33_bdx,27,4.637250,0.434874,13,9.951923,0.270216,3.461539,0.004239,40,nuit,False,True,64,0,0,0,0,0,0


### Enregistrement des données preprocessed

In [45]:
df_imputed.to_parquet(
    "/content/drive/MyDrive/nexialog/data_preprocessed_S1.parquet",
    engine="pyarrow",
    index=False
)

# agrégation et feature engenering

In [ ]:
donnees_preprocessees = pd.read_parquet(chemin_donnees_preprocessees, engine="pyarrow").sort_values(by="date_hour")